#Function Definitions (Do First)

In [2]:
from PIL import Image
import numpy as np

In [35]:
def loadPaletteSheet(colorSheet, format=0):
  length = 64
  height = 4
  topSpacing = 8
  numPalettes = 4
  paletteTypes = 4
  typeSpace = 29

  img = Image.open(colorSheet)
  arr = np.asarray(img)

  palettes = []

  for i in range(paletteTypes):
    for k in range(numPalettes):
      for h in range(height):
        for l in range(length):
          color = arr[(i * (typeSpace) + k * (height + 1) + h + topSpacing), l]

          r = color[0]
          g = color[1]
          b = color[2]
          a = color[3]

          argbd = a << 24 | r << 16 | g << 8 | b
          argbh = '{:02x}{:02x}{:02x}'.format(r, g, b)

          if(format == 0):
            palettes.append(argbd)
          else:
            palettes.append(argbh)

  return palettes

In [4]:
def findColor(color, palette):
  r = color[0]
  g = color[1]
  b = color[2]
  a = color[3]

  argbd = a << 24 | r << 16 | g << 8 | b
  # argbh = '0x{:02x}{:02x}{:02x}{:02x}'.format(a, r, g, b)

  if(argbd in palette):
    return palette.index(argbd)
  else:
    return 0

In [5]:
def getSpritePaletteIndexed(sprite, sizeH, sizeW, palette):
  # print(np.shape(sprite))
  indexedSprite = np.zeros((sizeH, sizeW), dtype=np.uint8)
  for i in range(sizeH):
    for k in range(sizeW):
      indexedSprite[i, k] = findColor(sprite[i,k], palette)
  return indexedSprite

In [6]:
def spritesToArray(infile, palette, sizeH, sizeW, rows, cols):
    img = Image.open(infile)
    arr = np.asarray(img)

    flattenedSprites = np.zeros((rows*cols, sizeW*sizeH), dtype=np.uint8);

    for i in range(rows):
      for k in range(cols):
        sprite = arr[
         (i * (sizeH + 1)) + 1 : (i * (sizeH + 1)) + 1 + sizeH,
         (k * (sizeW + 1)) + 1 : (k * (sizeW + 1)) + 1 + sizeW]

        spi = getSpritePaletteIndexed(sprite, sizeH, sizeW, palette)
        flattenedSprites[(i*cols) + k] = spi.flatten()

    flattenedSprites = flattenedSprites.flatten()
    return flattenedSprites

In [7]:
def spritesToFile(colorSheet, inputB1, inputB2, inputB3, inputB4, inputL, inputM, inputS, output):
    palettes = loadPaletteSheet(colorSheet)

    paletteB = palettes[0 : 256 * 4 - 1]
    paletteL = palettes[256 * 4 : 256 * 8 - 1]
    paletteM = palettes[256 * 8 : 256 * 12 - 1]
    paletteS = palettes[256 * 12 : 256 * 16 - 1]

    spriteL = spritesToArray(inputL, paletteL, 64, 64, 8, 8)
    spriteM = spritesToArray(inputM, paletteM, 32, 32, 8, 8)
    spriteS = spritesToArray(inputS, paletteS, 16, 16, 16, 16)

    spriteB1 = spritesToArray(inputB1, paletteB, 288, 512, 1, 1)
    spriteB2 = spritesToArray(inputB2, paletteB, 288, 512, 1, 1)
    spriteB3 = spritesToArray(inputB3, paletteB, 288, 512, 1, 1)
    spriteB4 = spritesToArray(inputB4, paletteB, 288, 512, 1, 1)

    print(spriteM[0:32*32])

    spriteData = []

    for i in range(len(palettes)):
      spriteData.append(palettes[i])

    for i in range(np.shape(spriteL)[0]):
      spriteData.append(spriteL[i])

    for i in range(np.shape(spriteM)[0]):
      spriteData.append(spriteM[i])

    for i in range(np.shape(spriteS)[0]):
      spriteData.append(spriteS[i])

    for i in range(np.shape(spriteB1)[0]):
      spriteData.append(spriteB1[i])

    for i in range(np.shape(spriteB2)[0]):
      spriteData.append(spriteB2[i])

    for i in range(np.shape(spriteB3)[0]):
      spriteData.append(spriteB3[i])

    for i in range(np.shape(spriteB4)[0]):
      spriteData.append(spriteB4[i])

    f = open(output, "w")
    f.write("#ifndef SPRITES_DATA\n#define SPRITES_DATA\n\n")
    f.write("uint32_t SPRITE_DATA[] = {");


    for i in range(len(spriteData)):
      f.write(str(spriteData[i]))
      if(i < len(spriteData) - 1):
        f.write(",")

    f.write("};\n\n#endif\n");


In [8]:
def createSpriteSheet(name, sizeH, sizeW, rows, cols):

  spriteSheet = np.zeros((
    (sizeH + 1) * rows + 1,
    (sizeW + 1) * cols + 1, 4), dtype=np.uint8)

  print(np.shape(spriteSheet))

  for i in range(rows):
    spriteSheet[i * (sizeH + 1),:,3] = 255

  for i in range(cols):
    spriteSheet[:, i * (sizeW + 1),3] = 255

  spriteSheet[(sizeH + 1) * rows,:,3] = 255
  spriteSheet[:,(sizeW + 1) * cols,3] = 255

  print(np.shape(spriteSheet))

  im = Image.fromarray(spriteSheet)
  im.save(name)

# Create Sprite Sheets (Don't Do)


In [39]:
l = 64
h = 4
spacing = 9
numPalettes = 4

colorSheet = np.zeros(((h * (numPalettes + 1) + spacing) * 4, l, 4), dtype=np.uint8)

for i in range(4):
  colorSheet[i * (h * (numPalettes + 1) + spacing) + 7, :, 3] = 255
  colorSheet[i * (h * (numPalettes + 1) + spacing) + 12, :, 3] = 255
  colorSheet[i * (h * (numPalettes + 1) + spacing) + 17, :, 3] = 255
  colorSheet[i * (h * (numPalettes + 1) + spacing) + 22, :, 3] = 255
  colorSheet[i * (h * (numPalettes + 1) + spacing) + 27, :, 3] = 255

im = Image.fromarray(colorSheet)
im.save("BlankColorSheet.png")

(116, 64, 4)


In [31]:
# Create Sprite Sheets

createSpriteSheet("bgSprite1.png", 288, 512, 1, 1)
createSpriteSheet("bgSprite2.png", 288, 512, 1, 1)
createSpriteSheet("bgSprite3.png", 288, 512, 1, 1)
createSpriteSheet("bgSprite4.png", 288, 512, 1, 1)
createSpriteSheet("largeSpriteSheet.png", 64, 64, 8, 8)
createSpriteSheet("mediumSpriteSheet.png", 32, 32, 8, 8)
createSpriteSheet("smallSpriteSheet.png", 16, 16, 16, 16)

(290, 514, 4)
(290, 514, 4)
(290, 514, 4)
(290, 514, 4)
(290, 514, 4)
(290, 514, 4)
(290, 514, 4)
(290, 514, 4)
(521, 521, 4)
(521, 521, 4)
(265, 265, 4)
(265, 265, 4)
(273, 273, 4)
(273, 273, 4)


# Load Palette To Pixilart (Optional)

In [36]:
palettes = loadPaletteSheet('samplePalette.png', 1)
paletteTypes = ["Background", "Large", "Medium", "Small"]
print(len(palettes))
for i in range(4):
  print(paletteTypes[i])
  for k in range(4):
    paletteStr = ""
    for l in range(256):
      paletteStr += palettes[(((i * 4) + k) * 256) + l] + ","
    print(paletteStr)

4096
Background
000000,ffffff,d4d4d4,a1a1a1,787878,545454,303030,000000,edc5c5,e68383,ff0000,de2424,ad3636,823737,592b2b,f5d2ee,eb8dd7,f700b9,bf1f97,9c277f,732761,4f2445,e2bcf7,bf79e8,9d00ff,8330ba,6d3096,502c69,351b47,c5c3f0,736feb,0905f7,2e2eb0,2d2d80,252554,090936,c7e2ed,6ac3e6,00bbff,279ac4,347c96,2d5b6b,103947,bbf0d9,6febb3,00ff88,2eb878,349166,2b694c,0c3d25,c2edc0,76ed70,0dff00,36c72c,408c3b,315c2e,144511,d6edbb,b5eb73,8cff00,89c93a,6f8f44,4b632a,2a400c,f1f2bf,eef069,ffff00,baba30,91913f,5e5e2b,3b3b09,ffdeb8,f2ae61,ff8400,c48037,85623d,573e25,3d2309,fcbbae,ff8066,ff2b00,cc553d,9c5b4e,61372e,36130b,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,000000,0000

# Save Sprites To H File (Do Last)

In [ ]:
# Modify sprite sheets
# recommended site: https://www.pixilart.com/draw#

In [59]:
spritesToFile('defaultPaletteSheet.png',
              'bgSprite1.png',
              'bgSprite2.png',
              'bgSprite3.png',
              'bgSprite4.png',
              'largeSpriteSheet.png',
              'mediumSpriteSheet.png',
              'smallSpriteSheet.png',
              "spriteData.h")

[0 0 0 ... 0 0 0]
